Place for a picture

# Logistic Regression of a... (Phase Three Project)

## Business Problem/Question

## EDA

In [2]:
#Importing packages
import numpy as np
import pandas as pd 
import math

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.dummy import DummyClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, plot_roc_curve
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

## Read in and create new csv

In [79]:
#Read in data
df_vehicles = pd.read_csv("Data\Traffic_Crashes_-_Vehicles_20231109.csv", parse_dates=["CRASH_DATE"], low_memory=False)
df_people = pd.read_csv("Data\Traffic_Crashes_-_People_20231109.csv", parse_dates=["CRASH_DATE"], low_memory=False)
df_crashes = pd.read_csv("Data\Traffic_Crashes_-_Crashes_20231109.csv", parse_dates=["CRASH_DATE"], low_memory=False)

In [82]:
#Filter out any data from before 2021
df_crashes = df_crashes[df_crashes["CRASH_DATE"].dt.year >= 2021]
df_people = df_people[df_people["CRASH_DATE"].dt.year >= 2021]
df_vehicles = df_vehicles[df_vehicles["CRASH_DATE"].dt.year >= 2021]

In [83]:
#merge into one dataset
df = df_vehicles.merge(df_people, on=["CRASH_RECORD_ID", "CRASH_DATE", "RD_NO", "VEHICLE_ID"])
df = df.merge(df_crashes, on=["CRASH_RECORD_ID", "CRASH_DATE", "RD_NO"])

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 679292 entries, 0 to 679291
Columns: 144 entries, CRASH_UNIT_ID to LOCATION
dtypes: datetime64[ns](1), float64(23), int64(8), object(112)
memory usage: 751.5+ MB


In [85]:
df.describe()

,CRASH_UNIT_ID,UNIT_NO,NUM_PASSENGERS,VEHICLE_ID,VEHICLE_YEAR,OCCUPANT_CNT,CMV_ID,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,...,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
count,6.792920e+05,679292.000000,219414.000000,6.644880e+05,561242.000000,664488.000000,7031.000000,585.000000,20.000000,861.000000,...,679292.000000,679292.000000,679292.000000,679292.000000,679292.0,679292.000000,679292.000000,679292.000000,673578.000000,673578.000000
mean,1.362200e+06,1.497006,1.890217,1.293703e+06,2014.720349,1.607301,14835.690656,49.870085,48.250000,51.344948,...,0.030209,0.179204,0.100011,2.491577,0.0,13.439062,4.132498,6.438353,41.851968,-87.673570
std,1.965467e+05,0.574281,2.206709,1.882725e+05,108.667179,1.536657,1977.935756,31.714267,27.720171,23.192033,...,0.227039,0.616471,0.448486,1.817067,0.0,5.542364,1.998372,3.228507,0.352692,0.719074
min,1.017880e+06,0.000000,1.000000,9.647270e+05,1900.000000,0.000000,11324.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,-87.936193
25%,1.192216e+06,1.000000,1.000000,1.130732e+06,2009.000000,1.000000,13154.500000,48.000000,28.000000,38.000000,...,0.000000,0.000000,0.000000,2.000000,0.0,10.000000,2.000000,4.000000,41.780164,-87.722591
50%,1.362675e+06,1.000000,1.000000,1.293936e+06,2014.000000,1.000000,14861.000000,53.000000,53.000000,53.000000,...,0.000000,0.000000,0.000000,2.000000,0.0,14.000000,4.000000,6.000000,41.871165,-87.675493
75%,1.533972e+06,2.000000,2.000000,1.457914e+06,2018.000000,2.000000,16545.500000,53.000000,60.000000,65.000000,...,0.000000,0.000000,0.000000,3.000000,0.0,17.000000,6.000000,9.000000,41.922751,-87.633827
max,1.699577e+06,12.000000,46.000000,1.617697e+06,9999.000000,47.000000,18191.000000,740.000000,123.000000,250.000000,...,10.000000,19.000000,11.000000,48.000000,0.0,23.000000,7.000000,12.000000,42.022780,0.000000


In [86]:
df.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,1554880,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,2023-04-14 14:05:00,1,DRIVER,NaN,1478881.0,NaN,FORD,...,0.0,1.0,2.0,0.0,14,6,4,41.885256,-87.667091,POINT (-87.667090771055 41.88525590335)
1,1554881,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,2023-04-14 14:05:00,2,DRIVER,1.0,1478892.0,NaN,ISUZU,...,0.0,1.0,2.0,0.0,14,6,4,41.885256,-87.667091,POINT (-87.667090771055 41.88525590335)
2,1554881,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,2023-04-14 14:05:00,2,DRIVER,1.0,1478892.0,NaN,ISUZU,...,0.0,1.0,2.0,0.0,14,6,4,41.885256,-87.667091,POINT (-87.667090771055 41.88525590335)
3,1635369,af3481ea9ffe80af8e5b80c0f1a7d72c58e84fcf0822c7...,JG375671,2023-08-09 09:52:00,1,DRIVER,NaN,1556218.0,NaN,HYUNDAI,...,0.0,1.0,1.0,0.0,9,4,8,41.900574,-87.667270,POINT (-87.667269556509 41.900574013597)
4,1635370,af3481ea9ffe80af8e5b80c0f1a7d72c58e84fcf0822c7...,JG375671,2023-08-09 09:52:00,2,DRIVER,NaN,1556221.0,NaN,MAZDA,...,0.0,1.0,1.0,0.0,9,4,8,41.900574,-87.667270,POINT (-87.667269556509 41.900574013597)


In [87]:
df.describe(include="all")

<ipython-input-87-7150bae0ffd8>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include="all")


,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
count,6.792920e+05,679292,669194,679292,679292.000000,679258,219414.000000,6.644880e+05,13884,664488,...,679292.000000,679292.000000,679292.000000,679292.0,679292.000000,679292.000000,679292.000000,673578.000000,673578.000000,673578
unique,NaN,311358,306821,206202,NaN,8,NaN,NaN,2,685,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156030
top,NaN,a736767eac48b46f7ff9eea9f13c0f244a6a10b2d28bc1...,JG308994,2023-06-20 18:23:00,NaN,DRIVER,NaN,NaN,Y,CHEVROLET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-87.905309125103 41.976201139024)
freq,NaN,156,156,156,NaN,654677,NaN,NaN,8118,75620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1238
first,NaN,NaN,NaN,2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2023-11-09 02:40:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.362200e+06,NaN,NaN,NaN,1.497006,NaN,1.890217,1.293703e+06,NaN,NaN,...,0.179204,0.100011,2.491577,0.0,13.439062,4.132498,6.438353,41.851968,-87.673570,NaN
std,1.965467e+05,NaN,NaN,NaN,0.574281,NaN,2.206709,1.882725e+05,NaN,NaN,...,0.616471,0.448486,1.817067,0.0,5.542364,1.998372,3.228507,0.352692,0.719074,NaN
min,1.017880e+06,NaN,NaN,NaN,0.000000,NaN,1.000000,9.647270e+05,NaN,NaN,...,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,-87.936193,NaN
25%,1.192216e+06,NaN,NaN,NaN,1.000000,NaN,1.000000,1.130732e+06,NaN,NaN,...,0.000000,0.000000,2.000000,0.0,10.000000,2.000000,4.000000,41.780164,-87.722591,NaN


In [88]:
df.to_csv("Data\chicago_traffic_accidents_2021_to_11-09-2023.csv")

## After new csv made

## Feature Engineering

## Decide Xs/y

In [4]:
X = df.drop("", axis=1)
y = df[""]

KeyError: "[''] not found in axis"

## Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2024)

## Preprocessing Steps (SS, OHE, SI)

## Modeling (look at Coefficients, P-values)

## Evaluation OF/UF report Test

# Model 2

# Model 3